In [1]:
import pandas as pd
import requests
import json
import seaborn as sns

In [2]:
import matplotlib.pyplot as plt

In [3]:
mt150 = pd.read_csv('C:/Users/Aaron/Documents/ML_Udacity/PPP/All Data by State/All Data by State/150k plus/PPP Data 150k plus.csv')

In [4]:
print("Data Set: PPP More than $150k \nRows: {}\nCols: {}".format(mt150.shape[0], mt150.shape[1]))

Data Set: PPP More than $150k 
Rows: 661218
Cols: 16


In [16]:
# mt150.head()

In [6]:
# Creates a new column 'NAICSCode_int' which converts 'NAICSCode' into integer.

mt150['NAICSCode_int'] = mt150['NAICSCode'].fillna('0').astype('int')

In [7]:
def get_naics(naicscode):
    '''Requests NAICS api to retrieve code title.'''
    
    page = requests.get("http://api.naics.us/v0/q?year=2012&code={}".format(naicscode))
    naicspage = json.loads(page.content)
    try:
        title = naicspage['title']
    except:
        title = 'NAICS Not Found'
    return title

In [ ]:
## get_naics(mt150['NAICSCode_int'][0])

In [ ]:
# Retrieve all NAICS Titles

# mt150['NAICS_title'] = mt150['NAICSCode_int'].apply(lambda x: get_naics(x))

In [8]:
%%time
top50try = mt150['NAICSCode_int'][:50].apply(lambda x: get_naics(x))

Wall time: 5.86 s


In [9]:
print('Would take {} hours to retrieve all NAICS titles.'.format(((mt150.shape[0] / 50) * 5.29) /3600))

Would take 19.432462333333334 hours to retrieve all NAICS titles.


In [ ]:
# mt150.groupby('NAICSCode_int')['BusinessName'].nunique().sort_values(ascending=False)[:20]

In [10]:
# Create new frame with only unique NAICS codes.

uqNAICSCodes = pd.DataFrame(mt150['NAICSCode_int'].unique(), columns=['NAICSCode_int'])

In [11]:
%%time
# Retrieve titles for Unique NAICS codes.

uqNAICSCodes['titles'] = uqNAICSCodes['NAICSCode_int'].apply(lambda x: get_naics(x))

Wall time: 2min 42s


In [12]:
# Merge Unique titles into main frame. 

mt150 = mt150.merge(uqNAICSCodes, on='NAICSCode_int')

In [13]:
# Split LoanRange into two: LoanCategory and LoanRange

mt150[['LoanCategory', 'LoanRange']] = mt150['LoanRange'].str.split(' ', 1, expand=True)

In [18]:
pd.DataFrame(mt150.groupby('titles')[['BusinessName', 'LoanCategory']].count().sort_values(by='BusinessName', ascending=False))[:25]

,BusinessName,LoanCategory
titles,,
Full-Service Restaurants,33608,33608
Offices of Physicians (except Mental Health Specialists),20932,20932
NAICS Not Found,15674,15675
Offices of Lawyers,14310,14310
"Plumbing, Heating, and Air-Conditioning Contractors",13140,13140
New Car Dealers,12694,12694
Commercial and Institutional Building Construction,10942,10942
Religious Organizations,10681,10681
Offices of Dentists,10627,10627


In [19]:
catplot_frame = pd.DataFrame(mt150.groupby(['titles', 'LoanRange']).agg({'BusinessName': 'count'}))
catplot_frame.head()

BusinessName
titles                         LoanRange                       
Abrasive Product Manufacturing $1-2 million                   7
                               $150,000-350,000              42
                               $2-5 million                   7
                               $350,000-1 million            31
                               $5-10 million                  1

In [21]:
# sns.catplot(x = "LoanRange", y = "BuinessName", col = "titles")

In [15]:
# mt150.head()

In [ ]:
# mt150.loc[mt150['City'] == 'NEW ORLEANS']

In [ ]:
## Francisco recommends:
##    folium